In [1]:
from pathlib import Path
import typing
import json

In [2]:
path_to_savegame = Path(r'C:\Users\Victor\Documents\Paradox Interactive\Crusader Kings II\save games\Hriscu7')
save_game_name = '769_08_20.ck2'
complete_path = path_to_savegame / save_game_name

In [3]:
from ck2_savefile.parser import SaveFileParser


save_game_parser = SaveFileParser(file_path=complete_path)

In [4]:
response = save_game_parser.parse_data()

In [5]:
from ck2_savefile.search_type import DictSearch,OneLineKeyValueSearch,OptionalKeyDictSearch


search_dict = DictSearch(
    search_key = 'player',
    get_value_flag= False
)
character_dict_search = DictSearch(
    search_key='player_portrait',
    get_value_flag = False
)
search_key = OneLineKeyValueSearch(
    search_key = 'health'
)
optional_dicts = OptionalKeyDictSearch(
    search_key=None
)
response2 = response.get_by_search_term(search_dict)
response3 = response.get_by_search_term(character_dict_search)

In [6]:
player_dict : DictSearch.item_info_type = next(response2.generator)
player_portrait : DictSearch.item_info_type = next(response3.generator)

In [7]:
changes = player_dict.extend(
    input = {'data_key': 'caca',
             'data_value' : 'maca'
             },
    class_type= OneLineKeyValueSearch.item_info_type
)
changes2 = player_dict.extend(
    input = {'data_key': 'caca',
             'data_value' : 'maca'
             },
    class_type= OneLineKeyValueSearch.item_info_type
) 
changes3 = player_portrait.extend(
    input = {'data_key': 'caca',
             'data_value' : 'maca'
             },
    class_type= OneLineKeyValueSearch.item_info_type
)
changes4 = player_portrait.extend(
    input = {'data_key': 'caca',
             'data_value' : 'maca'
             },
    class_type= OneLineKeyValueSearch.item_info_type
)

In [8]:
from ck2_savefile.editor import EditorHandler

editor = EditorHandler(
    changes= [changes , changes2 , changes3,changes4],
    file_path= complete_path
)
editor.apply_changes()
editor.write_to_file(Path(r'C:\Users\Victor\Documents\GitHub\ck2savefile\dada.ck2'))